<a href="https://colab.research.google.com/github/Nikolay1982Nikolaev/Big-Data-Bicocca/blob/main/8_SA_with_embeddings_and_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
file_path = "/content/drive/MyDrive/00.Last_sprint_UNIVERSITA/06.BigData/"

In [4]:
import pandas as pd

trip_reviews = pd.read_csv(file_path+"data/tripadvisor_binary.csv")
print(f"review: {trip_reviews.iloc[0]['review']}")
print()
print(f"sentiment: {trip_reviews.iloc[0]['bin_y']}")

review: ok_nothing_special charge diamond_member hilton decided chain shot 20th anniversary seattle start booked_suite paid_extra website description suite bedroom_bathroom standard_hotel room_took printed reservation_desk showed said thing_like tv couch ect desk_clerk told oh mixed suite description kimpton website sorry free_breakfast got kidding embassy suit sitting_room bathroom_bedroom unlike kimpton call suite day_stay offer correct false_advertising send kimpton preferred_guest website email asking failure provide suite advertised_website reservation description furnished hard copy reservation printout website desk_manager duty reply solution send email trip guest survey follow email mail guess tell concerned guest staff ranged indifferent helpful_asked desk good_breakfast spot neighborhood hood told hotel gee best breakfast spot seattle block_away convenient hotel_doe know exist arrived_late night 11_pm inside run bellman busy chating cell_phone help_bag prior_arrival emailed_h

## Data Preprocessing

In [5]:
!pip install bert

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for bert: filename=bert-2.2.0-py3-none-any.whl size=3744 sha256=2578621a30f8667d785bd3f27f7c06e0c2eda7da0245739964153837c81fcf4c
  Stored in directory: /root/.cache/pip/wheels/d0/97/c8/5775d57c323970511488d9b53ce66cf0ded394fe4b4f6e0afb
  Created wheel for erlastic: filename=erlastic-2.0.0-py3-none-any.whl size=6780 sha256=369abb8f614f07c142b7f231382bb9ab60927169340eed1ace0807ed7a053e53
  Stored in directory: /root/.cache/pip/wheels/f5/ca/b1/ce71c2e0e9e0df3786697d06d53574be03ef86cd4fb0e2cb99
Successfully built bert erlastic


In [6]:
# Regular imports
import numpy as np
import pandas as pd
import tqdm # for progress bar
import math
import random
import re

# Tensorflow Import
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

# Bert Import for Tokenizer
import bert

## remove html leftover and other text impurities using regular expressions

In [8]:
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"s+[a-zA-Z]s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r's+', ' ', sentence)

    return sentence

# Clean all Reviews in DataFrame
reviews = []
sentences = list(trip_reviews['review'])
for sen in sentences:
    reviews.append(preprocess_text(sen))

In [9]:
# Save sentiments as dependent variable y
y = trip_reviews['bin_y']

tokenize the text and create a dictionary

In [10]:
!pip install transformers

In [11]:
# Load Tokenizer and Model
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [12]:
# Try Tokenizer
tokenizer.tokenize("That's an example")

['That', "'", 's', 'an', 'example']

In [13]:
# Try tokenizer to dictionary
tokenizer.convert_tokens_to_ids(tokenizer.tokenize("Thats an example"))

[1337, 1116, 1126, 1859]

tokenize your sentences

In [14]:
# Tokenize all reviews
def tokenize_reviews(text_reviews):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_reviews))

tokenized_reviews = [tokenize_reviews(review) for review in tqdm.tqdm(reviews)]

# Create list of list with review, sentiment, and length of review for each entry
reviews_with_len = [[review, y[i], len(review)] for i, review in tqdm.tqdm(enumerate(tokenized_reviews))]

# Shuffle dataset
random.shuffle(reviews_with_len)

# Sort by length of review
reviews_with_len.sort(key=lambda x: x[2])

# Drop review
sorted_reviews_labels = [(review_lab[0], review_lab[1]) for review_lab in tqdm.tqdm(reviews_with_len)]

100%|██████████| 5398/5398 [00:20<00:00, 261.06it/s]
5398it [00:00, 251668.50it/s]
100%|██████████| 5398/5398 [00:00<00:00, 560375.54it/s]


store the data in a TF dataset for more effiecient processing

In [15]:
# Create Tensorflow datasets
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_reviews_labels, output_types=(tf.int32, tf.int32))

# Define batch size and cut datasets by batch size
BATCH_SIZE = 16
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

Instructions for updating:
Use output_signature instead


## create train and test split

In [16]:
TOTAL_BATCHES = math.ceil(len(sorted_reviews_labels) / BATCH_SIZE)
TEST_BATCHES = TOTAL_BATCHES // 5
batched_dataset.shuffle(TOTAL_BATCHES)
test_data = batched_dataset.take(TEST_BATCHES)
train_data = batched_dataset.skip(TEST_BATCHES)

## Create the BERT model

Define the model

In [17]:
class TEXT_MODEL(tf.keras.Model):

    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=128,
                 cnn_filters=50,
                 dnn_units=512,
                 model_output_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="text_model"):
        super(TEXT_MODEL, self).__init__(name=name)

        self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        self.cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
        self.pool = layers.GlobalMaxPool1D()

        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=model_output_classes,
                                           activation="softmax")

    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l)
        l_1 = self.pool(l_1)
        l_2 = self.cnn_layer2(l)
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(l)
        l_3 = self.pool(l_3)

        concatenated = tf.concat([l_1, l_2, l_3], axis=-1) # (batch_size, 3 * cnn_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)

        return model_output

set hyperparameters

In [18]:
# Hyperparameters
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 200
CNN_FILTERS = 100
DNN_UNITS = 256
OUTPUT_CLASSES = 2
DROPOUT_RATE = 0.2
NB_EPOCHS = 2

# Build Model
text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)

# Compile Model
text_model.compile(loss="binary_crossentropy",
                       optimizer="adam",
                       metrics=["accuracy"])

...and finally, train and test the model

In [19]:
text_model.fit(train_data, epochs=NB_EPOCHS)

Epoch 1/2
271/271 [==============================] - 77s 272ms/step - loss: -55669.9414 - accuracy: 0.6026
Epoch 2/2
271/271 [==============================] - 45s 166ms/step - loss: -2630850.7500 - accuracy: 0.6052


In [20]:
text_model.evaluate(test_data)

67/67 [==============================] - 1s 11ms/step - loss: -6793903.0000 - accuracy: 0.5560


[-6793903.0, 0.5559701323509216]

In [21]:
from sklearn.metrics import classification_report
predictions = text_model.predict(test_data)
y_pred = np.around(predictions)
y_true = tf.concat([y for x, y in test_data], axis=0)
print(classification_report(y_pred, y_true))

67/67 [==============================] - 1s 18ms/step
              precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00         0
         0.0       1.00      0.56      0.71      1072

    accuracy                           0.56      1072
   macro avg       0.50      0.28      0.36      1072
weighted avg       1.00      0.56      0.71      1072



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


we can create a single, manufactured review and test the model on it

In [22]:
# Test model on two individual reviews
test_reviews = ['What can I say, position, confort and wiew are terrific.',
                'I have been deceived by all the positive opinions about this hotel.']
test_reviews_with_len = [[tokenize_reviews(test_reviews[0]),
                          1,
                          len(tokenize_reviews(test_reviews[0]))],
                         [tokenize_reviews(test_reviews[1]),
                          0,
                          len(tokenize_reviews(test_reviews[1]))]]
test_sorted_reviews_labels = [(review_lab[0], review_lab[1]) for review_lab in tqdm.tqdm(test_reviews_with_len)]

predict_input = tf.data.Dataset.from_generator(lambda: test_sorted_reviews_labels, output_types=(tf.int32, tf.int32))
BATCH_SIZE = 2
test_batched_dataset = predict_input.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

# Get model prediction
text_model.predict(test_batched_dataset)

100%|██████████| 2/2 [00:00<00:00, 2833.99it/s]

1/1 [==============================] - 0s 53ms/step


array([[0.],
       [0.]], dtype=float32)